### Introduction

During my PhD days at Boston University, I never bothered to learn how to log errors in Python. When I was testing my code on my local computer, I could simply see the errors on my terminal screen and when I would send my production code to BU's shared computing cluster, it would store all stderr messages in a file as long as I specify it on my <a href="https://www.bu.edu/tech/support/research/system-usage/running-jobs/submitting-jobs/">qsub job script</a>.


I later realized that it's not true for all cloud computing systems. In particular, when I started running my production code on Amazon Web Server, my python code could crash without leaving any error message. 

My first attempt was to use python's `try except` block to save the error message. 

### Simple Example 

In [1]:
y=3
try:
    x=2/0
    z=x+y
except Exception as e:
    print(e)

division by zero


You can see that it doesn't save the source of the errors (i.e. traceback error). For a code that runs hundreds of lines, we would like to know which line triggered the error message. This forced me to look better ways to log my errors on my own and then I stumbled onto python's `logging` class. It is an extremely powerful and customizable class. It takes some time to learn how to use it but I feel it's worth the investment. Let's start with a simple example.

### First logging event tracking

In [4]:
import logging
logging.basicConfig(filename='example.log')#, level=logging.DEBUG)
logging.debug('This message should be recorded if you are debugging')
logging.info("Hello there")
logging.warning('Also this')
logging.error('And this.')
logging.critical("Woahhh")

In [5]:
!cat example.log

ERROR:root:And this.
CRITICAL:root:Woahhh


Note how the error log contains three things: 
1. <b> What's the level of error? </b> Python has five levels of events that `logging` can track (in increasing severity): DEBUG, INFO, WARNING, ERROR, CRITICAL. By default, Python tracks WARNING and higher levels. This is why `logging.debug` and `logging.info` messages were written to file.
2. <b> Who wrote the error message? </b> This can be customized to track error messages coming from different modules or users
3. <b> What is the message? </b> Apart from storing explicit message, you can also save `Traceback` error using `exc_info=True` argument (see below).

Now let's go back to our first example.

### Use logging to save traceback message

In [6]:
y=3
try:
    x=2/0
    z=x+y
except Exception as e:
    logging.error("Exception occurred", exc_info=True)

In [7]:
!cat example.log

ERROR:root:And this.
CRITICAL:root:Woahhh
ERROR:root:Exception occurred
Traceback (most recent call last):
  File "<ipython-input-6-7980dd2c71e5>", line 3, in <module>
    x=2/0
ZeroDivisionError: division by zero


Note that we had to say `exc_info=True` so that it saves the Traceback of error.

Next we would learn how to customize the format of the message so that we can save the date and time of the message. Note that you need to restart your jupyter notebook as `basicConfig` is configured only once during a run, which is why it should be done at the beginning of your code.

In [1]:
import logging
logging.basicConfig(filename='example.log',format='%(asctime)s -%(levelname)s -%(message)s', datefmt='%d-%b-%y %H:%M:%S')
logging.warning('Admin logging in')

In [2]:
!cat example.log

ERROR:root:And this.
CRITICAL:root:Woahhh
ERROR:root:Exception occurred
Traceback (most recent call last):
  File "<ipython-input-6-7980dd2c71e5>", line 3, in <module>
    x=2/0
ZeroDivisionError: division by zero
18-Feb-22 08:15:19 -WARNING -Admin logging in


In [3]:
y=3
try:
    x=2/0
    z=x+y
except Exception as e:
    logging.error("Exception occurred", exc_info=True)

In [4]:
!cat example.log

ERROR:root:And this.
CRITICAL:root:Woahhh
ERROR:root:Exception occurred
Traceback (most recent call last):
  File "<ipython-input-6-7980dd2c71e5>", line 3, in <module>
    x=2/0
ZeroDivisionError: division by zero
18-Feb-22 08:15:19 -WARNING -Admin logging in
18-Feb-22 08:15:50 -ERROR -Exception occurred
Traceback (most recent call last):
  File "<ipython-input-3-7980dd2c71e5>", line 3, in <module>
    x=2/0
ZeroDivisionError: division by zero


This concludes an introduction to logging error. Here are the few references that can help you learn more:

https://realpython.com/python-logging/

https://docs.python.org/3/howto/logging.html